In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

In [68]:
#read data

sample_submission=pd.read_csv("../Data/sample_submission.csv")
train=pd.read_csv("../Data//train.csv")
test=pd.read_csv("../Data/test.csv")

In [69]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [70]:
train['Electrical']

0       SBrkr
1       SBrkr
2       SBrkr
3       SBrkr
4       SBrkr
        ...  
1455    SBrkr
1456    SBrkr
1457    SBrkr
1458    FuseA
1459    SBrkr
Name: Electrical, Length: 1460, dtype: object

In [71]:
y=train['SalePrice']
X=train.drop('SalePrice',axis=1)

In [72]:
print(X.shape, train.shape)

(1460, 80) (1460, 81)


In [73]:
X=X.dropna(axis=1)
test=test.dropna(axis=1)

In [74]:
print(X.shape, test.shape)

(1460, 61) (1459, 47)


In [75]:
list(set(X.columns)-set(test.columns))

['Exterior1st',
 'BsmtFinSF1',
 'SaleType',
 'BsmtFinSF2',
 'Utilities',
 'BsmtFullBath',
 'GarageCars',
 'Exterior2nd',
 'GarageArea',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'KitchenQual',
 'BsmtHalfBath',
 'MSZoning',
 'Functional']

In [76]:
X=X.drop(['TotalBsmtSF',
 'KitchenQual',
 'GarageCars',
 'Utilities',
 'Functional',
 'Exterior2nd',
 'Exterior1st',
 'BsmtFinSF1',
 'BsmtHalfBath',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'SaleType',
 'BsmtFullBath',
 'GarageArea',
 'MSZoning'],axis=1)

In [77]:
set(test.columns)-set(X.columns)

{'Electrical'}

In [78]:
test=test.drop(['Electrical'],axis=1)

In [79]:
print(test.shape, X.shape)

(1459, 46) (1460, 46)


In [80]:
categorical_columns=[col for col in X.columns if X[col].dtype=='object']
numerical_columns=[col for col in X.columns if X[col].dtype!='object']

In [81]:
categorical_columns

['Street',
 'LotShape',
 'LandContour',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'PavedDrive',
 'SaleCondition']

In [82]:
numerical_columns

['Id',
 'MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [83]:
X['Street'].value_counts(),test['Street'].value_counts()

(Street
 Pave    1454
 Grvl       6
 Name: count, dtype: int64,
 Street
 Pave    1453
 Grvl       6
 Name: count, dtype: int64)

In [84]:
from sklearn.preprocessing import OrdinalEncoder

In [85]:
enc=OrdinalEncoder()
ord_encoded=enc.fit_transform(X[categorical_columns])
ord_tested=enc.fit_transform(test[categorical_columns])


In [86]:
ord_encoded.shape,ord_tested.shape

((1460, 20), (1459, 20))

In [87]:
enc.get_feature_names_out(categorical_columns).shape

(20,)

In [88]:
ord_df=pd.DataFrame(ord_encoded,columns=enc.get_feature_names_out(categorical_columns))
ord_test=pd.DataFrame(ord_tested,columns=enc.get_feature_names_out(categorical_columns))
X_encoded=pd.concat([X,ord_df],axis=1)
test_encoded=pd.concat([test,ord_test],axis=1)
X_encoded=X_encoded.drop(categorical_columns,axis=1)
test_encoded=test_encoded.drop(categorical_columns,axis=1)

In [89]:
X_encoded.describe()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,10516.828082,6.099315,5.575342,1971.267808,1984.865753,1162.626712,346.992466,5.844521,...,0.613014,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,421.610009,42.300571,9981.264932,1.382997,1.112799,30.202904,20.645407,386.587738,436.528436,48.623081,...,0.644666,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,1.000000,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,334.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,365.750000,20.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,882.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,730.500000,50.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,1087.000000,0.000000,0.000000,...,1.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,1095.250000,70.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,1391.250000,728.000000,0.000000,...,1.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,1460.000000,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,4692.000000,2065.000000,572.000000,...,3.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [90]:
X_encoded[numerical_columns].describe()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,10516.828082,6.099315,5.575342,1971.267808,1984.865753,1162.626712,346.992466,5.844521,...,0.613014,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,421.610009,42.300571,9981.264932,1.382997,1.112799,30.202904,20.645407,386.587738,436.528436,48.623081,...,0.644666,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,1.000000,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,334.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,365.750000,20.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,882.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,730.500000,50.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,1087.000000,0.000000,0.000000,...,1.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,1095.250000,70.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,1391.250000,728.000000,0.000000,...,1.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,1460.000000,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,4692.000000,2065.000000,572.000000,...,3.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [91]:
X_encoded.head()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,8450,7,5,2003,2003,856,854,0,...,0,0,61,0,0,0,0,0,2,2008
1,2,20,9600,6,8,1976,1976,1262,0,0,...,1,298,0,0,0,0,0,0,5,2007
2,3,60,11250,7,5,2001,2002,920,866,0,...,1,0,42,0,0,0,0,0,9,2008
3,4,70,9550,7,5,1915,1970,961,756,0,...,1,0,35,272,0,0,0,0,2,2006
4,5,60,14260,8,5,2000,2000,1145,1053,0,...,1,192,84,0,0,0,0,0,12,2008


In [92]:
X_encoded.shape,test_encoded.shape

((1460, 26), (1459, 26))

In [93]:
from sklearn.preprocessing import MinMaxScaler

In [94]:
scaler=MinMaxScaler()
X_encoded_scaled=scaler.fit_transform(X_encoded)
test_encoded_scaled=scaler.fit_transform(test_encoded)

In [95]:
X_encoded_scaled.shape,X_encoded.shape,test_encoded_scaled.shape,test_encoded.shape

((1460, 26), (1460, 26), (1459, 26), (1459, 26))

## Model bulding

In [96]:
from sklearn.model_selection import train_test_split

In [97]:
X_train,X_valid,y_train,y_valid=train_test_split(X_encoded_scaled,y,test_size=0.2,random_state=1)

In [98]:
len(X_train),len(X_valid),len(y_train),len(y_valid)

(1168, 292, 1168, 292)

In [99]:
from sklearn.linear_model import LinearRegression

In [100]:
model_1=LinearRegression()

In [102]:
model_1.fit(X_train,y_train)

LinearRegression()

In [103]:
y_predict=model_1.predict(X_valid)

In [104]:
from sklearn.metrics import mean_absolute_error as mae

In [105]:
mae(y_predict,y_valid)

23344.054949826867

In [106]:
y_valid

258     231500
267     179500
288     122000
649      84500
1233    142000
         ...  
163     103200
47      249700
1432     64500
98       83000
409     339750
Name: SalePrice, Length: 292, dtype: int64

## 1.Feature Engineering
Polynomial Features

In [107]:
from sklearn.preprocessing import PolynomialFeatures

In [108]:
poly=PolynomialFeatures(degree=2,include_bias=False)
X_train_poly=poly.fit_transform(X_train)
X_valid_poly=poly.transform(X_valid)
test_poly=poly.transform(test_encoded_scaled)

In [109]:
model_1.fit(X_train_poly,y_train)
y_pred_1=model_1.predict(X_valid_poly)

In [111]:
mae(y_valid,y_pred_1)

250335406607832.38

## 2.Variance Threshold
Remove features with low variance, as they provide little information to the model.

In [112]:
from sklearn.feature_selection import VarianceThreshold

In [113]:
selector=VarianceThreshold(threshold=0.01)
X_train_var=selector.fit_transform(X_train)
X_valid_var=selector.transform(X_valid)
test_var=selector.transform(test_encoded_scaled)

In [114]:
model_1.fit(X_train_var,y_train)
y_pred_2=model_1.predict(X_valid_var)

In [115]:
mae(y_valid,y_pred_2)

27667.293263201405

In [116]:
from sklearn.ensemble import RandomForestRegressor

In [117]:
rf_model=RandomForestRegressor(random_state=1)
rf_model.fit(X_train,y_train)

RandomForestRegressor(random_state=1)

In [118]:
importances=rf_model.feature_importances_

#Sort features by importance
indices=np.argsort(importances)[::-1]

# print the feature ranking
for f in range(X_train.shape[1]):
    print(f"{f+1}. Feature {indices[f]}({importances[indices[f]]})")

1. Feature 3(0.5745931058227084)
2. Feature 10(0.1312716773726005)
3. Feature 7(0.07822397677179424)
4. Feature 2(0.03691791126538648)
5. Feature 5(0.03337051731105409)
6. Feature 8(0.02418437944431081)
7. Feature 6(0.019330913818016424)
8. Feature 15(0.015046999362626849)
9. Feature 18(0.01212848919991582)
10. Feature 24(0.011274764947038718)
11. Feature 17(0.010530875136004834)
12. Feature 0(0.010407482819570204)
13. Feature 11(0.010065878332357302)
14. Feature 4(0.007631518897744946)
15. Feature 16(0.005011225542724217)
16. Feature 1(0.004817969646883433)
17. Feature 25(0.003744139048476659)
18. Feature 13(0.0029061377481288285)
19. Feature 21(0.0028571496105006967)
20. Feature 12(0.0014386369548681412)
21. Feature 20(0.001417987098975621)
22. Feature 19(0.0012623676709773845)
23. Feature 14(0.0008483757437916279)
24. Feature 23(0.00025775512636287413)
25. Feature 9(0.0002492036911356695)
26. Feature 22(0.00021056161604524518)


In [119]:
n_top_features=30
top_features=indices[:n_top_features]

In [120]:
model_1.fit(X_train,y_train)
y_pred_3=model_1.predict(X_valid)

In [121]:
mae(y_valid,y_pred_3)

23344.054949826867

In [122]:
# Create polynomial features for selected features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly_rfe = poly.fit_transform(X_train)
X_valid_poly_rfe = poly.transform(X_valid)
test_poly_rfe=poly.transform(test_encoded_scaled)

# Train the linear regression model
model_1.fit(X_train_poly_rfe, y_train)

# Predict on the test set
y_pred_5 = model_1.predict(X_valid_poly_rfe)

# Calculate MAE
mae(y_valid,y_pred_5)

250335406607832.38

In [130]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [131]:
param_grid={'alpha':[0.1,1,10,100,1000]}

ridge=Ridge()
ridge_cv=GridSearchCV(ridge,param_grid,cv=5,scoring='neg_mean_absolute_error')
ridge_cv.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.1, 1, 10, 100, 1000]},
             scoring='neg_mean_absolute_error')

In [132]:
y_pred_ridge=ridge_cv.predict(X_valid)
mae_ridge=mae(y_valid,y_pred_ridge)
print(f"Ridge Regression MAE:{mae_ridge}")

Ridge Regression MAE:23413.656821703673


In [133]:
from sklearn.ensemble import GradientBoostingRegressor

#Train a Gradient Boosting Regressor
gbr=GradientBoostingRegressor()

gbr.fit(X_train,y_train)
y_pred_gbr=gbr.predict(X_valid)
mae_gbr=mae(y_valid,y_pred_gbr)

print(f"Gradient Boosting Regressor MAE:{mae_gbr}")

# Combine Prediction 
y_pred_combined=(y_pred_ridge+y_pred_gbr)/2
mae_combined=mae(y_valid,y_pred_combined)
print(f"Combined Model MAE:{mae_combined}")

Gradient Boosting Regressor MAE:16586.668573252522
Combined Model MAE:18102.50854336384


In [134]:
import xgboost as xgb

xgbr=xgb.XGBRegressor()
xgbr.fit(X_train,y_train)

#Predict on the valid set
y_pred_xgbr=xgbr.predict(X_valid)
mae_xgbr=mae(y_valid,y_pred_xgbr)
print(f"XGBoost Regressor MAE:{mae_xgbr}")

XGBoost Regressor MAE:17562.27565817637


In [135]:
from sklearn.model_selection import GridSearchCV

In [136]:
gbr.predict(test_encoded_scaled)

array([166734.13893108, 190944.20865739, 193974.66237364, ...,
       181759.86005289, 138789.41193764, 227448.42791637])

In [137]:
sample_submission= pd.read_csv('../Data/sample_submission.csv')
sample_submission['SalePrice']=gbr.predict(test_encoded_scaled)
sample_submission.to_csv('../Data/submission.csv',index=False)

In [138]:
sample_submission.head()

,Id,SalePrice
0,1461,166734.138931
1,1462,190944.208657
2,1463,193974.662374
3,1464,203963.702405
4,1465,216081.046263


In [139]:
X.head()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,1,60,8450,Pave,Reg,Lvl,Inside,Gtl,CollgCr,Norm,...,0,61,0,0,0,0,0,2,2008,Normal
1,2,20,9600,Pave,Reg,Lvl,FR2,Gtl,Veenker,Feedr,...,298,0,0,0,0,0,0,5,2007,Normal
2,3,60,11250,Pave,IR1,Lvl,Inside,Gtl,CollgCr,Norm,...,0,42,0,0,0,0,0,9,2008,Normal
3,4,70,9550,Pave,IR1,Lvl,Corner,Gtl,Crawfor,Norm,...,0,35,272,0,0,0,0,2,2006,Abnorml
4,5,60,14260,Pave,IR1,Lvl,FR2,Gtl,NoRidge,Norm,...,192,84,0,0,0,0,0,12,2008,Normal
